<a href="https://colab.research.google.com/github/mhsnur/TESIS-S2-Geomatika-FT-UGM-Muhsin-Nur-Alamsyah/blob/main/RANDOM_FOREST_JADI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

!pip install laspy cloth-simulation-filter numpy

import laspy
import numpy as np
import os
import CSF
from google.colab import drive


def csf_ground_non_ground(las_path,
                          cloth_resolution=2.0,
                          threshold=0.5,
                          rigidness=6,
                          slope_smooth=True,
                          save_ground_path=None,
                          save_non_ground_path=None):
    """
    Gunakan CSF untuk memisahkan ground dan non-ground dari point cloud.
    """

    las = laspy.read(las_path)
    points = las.points
    xyz = np.vstack((las.x, las.y, las.z)).T

    csf = CSF.CSF()

    csf.params.cloth_resolution = cloth_resolution
    csf.params.threshold = threshold
    csf.params.rigidness = rigidness
    csf.params.bSloopSmooth = slope_smooth

    csf.setPointCloud(xyz.tolist())

    ground = CSF.VecInt()
    non_ground = CSF.VecInt()

    csf.do_filtering(ground, non_ground)

    ground_idx = np.array(ground, dtype=int)
    non_ground_idx = np.array(non_ground, dtype=int)

    if save_ground_path:
        las_ground = laspy.create(point_format=las.header.point_format,
                                  file_version=las.header.version)
        las_ground.points = points[ground_idx]
        las_ground.header.offsets = las.header.offsets
        las_ground.header.scales = las.header.scales
        las_ground.write(save_ground_path)

    if save_non_ground_path:
        las_non_ground = laspy.create(point_format=las.header.point_format,
                                      file_version=las.header.version)
        las_non_ground.points = points[non_ground_idx]
        las_non_ground.header.offsets = las.header.offsets
        las_non_ground.header.scales = las.header.scales
        las_non_ground.write(save_non_ground_path)

    return ground_idx, non_ground_idx

path_base = '/content/drive/MyDrive/Segmented_LAS/DATA TESIS'
input_las = os.path.join(path_base, "DATA TESTING.las")
ground_output = os.path.join(path_base, "ground_csf.las")
nonground_output = os.path.join(path_base, "non_ground_csf.las")

ground_idx, non_ground_idx = csf_ground_non_ground(
    las_path=input_las,
    cloth_resolution=2.0,
    threshold=0.5,
    rigidness=6,
    slope_smooth=True,
    save_ground_path=ground_output,
    save_non_ground_path=nonground_output
)


print(f"Jumlah titik ground: {len(ground_idx)}")
print(f"Jumlah titik non-ground: {len(non_ground_idx)}")
print(f"✅ Ground disimpan di: {ground_output}")
print(f"✅ Non-ground disimpan di: {nonground_output}")


Jumlah titik ground: 7526793
Jumlah titik non-ground: 10785349
✅ Ground disimpan di: /content/drive/MyDrive/Segmented_LAS/DATA TESIS/ground_csf.las
✅ Non-ground disimpan di: /content/drive/MyDrive/Segmented_LAS/DATA TESIS/non_ground_csf.las


In [ ]:

from scipy.spatial import cKDTree

def extract_features(las_data, k_neighbors=10):
    xyz = np.vstack((las_data.x, las_data.y, las_data.z)).T
    rgb = np.vstack((las_data.red, las_data.green, las_data.blue)).T
    intensity = las_data.intensity

    tree = cKDTree(xyz)
    z_std_list = []
    for i in range(len(xyz)):
        _, idx = tree.query(xyz[i], k=k_neighbors)
        neighbors = xyz[idx]
        z_std = np.std(neighbors[:, 2])
        z_std_list.append(z_std)
    z_std_array = np.array(z_std_list)

    features = np.hstack((
        rgb,
        intensity.reshape(-1, 1),
        z_std_array.reshape(-1, 1)
    ))

    return features

veg_las = laspy.read(os.path.join(path_base, "VEGETASI_1.las"))
bdg_las = laspy.read(os.path.join(path_base, "BANGUNAN_1.las"))

features_veg = extract_features(veg_las)
features_bdg = extract_features(bdg_las)

labels_veg = np.full(len(features_veg), 5)
labels_bdg = np.full(len(features_bdg), 6)

X = np.vstack((features_veg, features_bdg))
y = np.hstack((labels_veg, labels_bdg))


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print("=== Evaluasi Model RF ===")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

las_all = laspy.read(input_las)
non_ground_points = laspy.create(point_format=las_all.header.point_format, file_version=las_all.header.version)
non_ground_points.points = las_all.points[non_ground_idx]

features_non_ground = extract_features(non_ground_points)
predicted_labels = clf.predict(features_non_ground)


final_labels = np.zeros(len(las_all.points), dtype=np.uint8)
final_labels[ground_idx] = 2
final_labels[non_ground_idx] = predicted_labels


final_las = laspy.create(point_format=las_all.header.point_format, file_version=las_all.header.version)
final_las.points = las_all.points
final_las.classification = final_labels
final_las.header.offsets = las_all.header.offsets
final_las.header.scales = las_all.header.scales

output_final = os.path.join(path_base, "hasil_segmentasi_rf.las")
final_las.write(output_final)

print(f"✅ Hasil klasifikasi final disimpan di: {output_final}")


=== Evaluasi Model RF ===
              precision    recall  f1-score   support

           5       1.00      0.99      0.99    624498
           6       0.99      1.00      1.00    835967

    accuracy                           0.99   1460465
   macro avg       0.99      0.99      0.99   1460465
weighted avg       0.99      0.99      0.99   1460465

[[619890   4608]
 [  3110 832857]]
✅ Hasil klasifikasi final disimpan di: /content/drive/MyDrive/Segmented_LAS/DATA TESIS/hasil_csf_rf_segmentasi.las
